# Preliminary Steps:

### Imports

In [1]:
# Imports 
import os
import pandas as pd
import numpy as np

### Create Data Folder

In [2]:
# Create 'Data' folder
os.makedirs('Data/',exist_ok=True)
os.listdir("Data/")

['.ipynb_checkpoints',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']

## Load Datasets

In [3]:
# Load 'akas' dataset
akas = pd.read_csv('Data/title-akas-us-only.csv', low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [4]:
# Load 'basics' dataset
basics = pd.read_csv('Data/title.basics.tsv.gz',sep='\t', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
# Load 'ratings' dataset
ratings = pd.read_csv('Data/title.ratings.tsv.gz', sep='\t', low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1989
1,tt0000002,5.8,265
2,tt0000003,6.5,1850
3,tt0000004,5.5,178
4,tt0000005,6.2,2635


## Filtering/Cleaning Steps:

### AKAs:

- keep only US movies.
- Replace "\N" with np.nan

In [6]:
# Keep only US movies.
# AKAs US only file already uploaded.

In [7]:
# Replace "\N" with np.nan
akas.replace({'\\N':np.nan}, inplace=True)

In [8]:
# Confirm changes made are permanent
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


### Title Basics:
 - Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
 - Replace "\N" with np.nan
 - Eliminate movies that are null for runtimeMinutes
 - Eliminate movies that are null for genre
 - keep only titleType==Movie
 - Convert the startYear column to float data type.
 - Filter the dataframe using startYear. Keep years between 2000-2021 (Including 2000 and 2021)
 - Eliminate movies that include "Documentary" in the genre (see tip below).

In [11]:
# Keep only US movies
keepers = basics['tconst'].isin(akas['titleId'])
keepers

0            True
1            True
2           False
3           False
4            True
            ...  
10064698    False
10064699    False
10064700    False
10064701    False
10064702    False
Name: tconst, Length: 10064703, dtype: bool

In [12]:
# Filter the basics table down to only include the US by using the filter akas dataframe
basics = basics[keepers]

In [13]:
# Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,NaN,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,NaN,1,"Short,Sport"


In [14]:
# Eliminate movies that are null for the 'runtimeMinutes' and 'genres' columns
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])

In [15]:
# Check value counts of 'titleType' column
basics['titleType'].value_counts()

short           278441
movie           203635
tvEpisode       173936
video           115674
tvSeries         36847
tvMovie          25959
tvSpecial         9295
tvMiniSeries      5900
tvShort           4100
videoGame          194
Name: titleType, dtype: int64

In [16]:
# Keep only titleType == movie
basics = basics[basics['titleType'] == "movie"]

In [17]:
# Convert the startYear column to float data type
basics['startYear'] = basics['startYear'].astype(float)

In [18]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203635 entries, 8 to 10064469
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          203635 non-null  object 
 1   titleType       203635 non-null  object 
 2   primaryTitle    203635 non-null  object 
 3   originalTitle   203635 non-null  object 
 4   isAdult         203635 non-null  object 
 5   startYear       200097 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  203635 non-null  object 
 8   genres          203635 non-null  object 
dtypes: float64(1), object(8)
memory usage: 15.5+ MB


In [21]:
# Filter dataf]rame using 'startYear' column
basics = basics.loc[(basics['startYear'] >= 2000.0)
                   & (basics['startYear'] <= 2021.0)]

In [22]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

### Ratings:
- Keep only movies that were included in your final title basics dataframe above.
(Use AKAs table, see "Filtering one dataframe based on another" section below)
- Replace "\N" with np.nan (if any)

In [26]:
# Filter dataframe using 'akas' dataframe
keep = ratings['tconst'].isin(akas['titleId'])
keep

0           True
1           True
2          False
3          False
4           True
           ...  
1333574    False
1333575    False
1333576    False
1333577    False
1333578    False
Name: tconst, Length: 1333579, dtype: bool

In [27]:
# Filter dataframe
ratings = ratings[keep]

In [28]:
# Replace "\N" with np.nan
ratings.replace({'\\N':np.nan}, inplace=True)
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1989
1,tt0000002,5.8,265
4,tt0000005,6.2,2635
5,tt0000006,5.0,183
6,tt0000007,5.4,825
...,...,...,...
1333540,tt9916200,8.1,232
1333541,tt9916204,8.2,266
1333548,tt9916348,8.3,18
1333549,tt9916362,6.4,5442


## Final DataFrame Tests:

In [23]:
# Final test of new 'akas' dataframe
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [10]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452564 entries, 0 to 1452563
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1452564 non-null  object
 1   ordering         1452564 non-null  int64 
 2   title            1452564 non-null  object
 3   region           1452564 non-null  object
 4   language         4018 non-null     object
 5   types            981678 non-null   object
 6   attributes       47016 non-null    object
 7   isOriginalTitle  1451222 non-null  object
dtypes: int64(1), object(7)
memory usage: 88.7+ MB


In [24]:
# Final test of new 'basics' dataframe
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34801,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61113,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67487,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67665,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86792,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [25]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81900 entries, 34801 to 10064469
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81900 non-null  object 
 1   titleType       81900 non-null  object 
 2   primaryTitle    81900 non-null  object 
 3   originalTitle   81900 non-null  object 
 4   isAdult         81900 non-null  object 
 5   startYear       81900 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81900 non-null  object 
 8   genres          81900 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


In [29]:
# Final test of new 'ratings' dataframe
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1989
1,tt0000002,5.8,265
4,tt0000005,6.2,2635
5,tt0000006,5.0,183
6,tt0000007,5.4,825


In [30]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504060 entries, 0 to 1333554
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         504060 non-null  object 
 1   averageRating  504060 non-null  float64
 2   numVotes       504060 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.4+ MB


## Save New Filtered DataFrames to Data Folder

In [31]:
# Save new dataframe to 'Data' folder.
akas.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)

In [32]:
# Save new dataframe to 'Data' folder.
basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)

In [33]:
# Save new dataframe to 'Data' folder.
ratings.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

### IMDB Data Sources:
- Downloads Page - https://developer.imdb.com/non-commercial-datasets/
- Data Dictionary - https://developer.imdb.com/non-commercial-datasets/
- IMDB Logo - https://www.themoviedb.org/about/logos-attribution

